In [1]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'datascience'
driver = GraphDatabase.driver(host,auth=(user, password))

In [9]:
PATH = r"C:\Users\Chai\.Neo4jDesktop\neo4jDatabases\database-a4494c5c-be01-475f-9407-58c9feb17dee\installation-4.2.1\import\MOCT_NODE_LATLNG.csv"

df = pd.read_csv(PATH)

print(df.head(10))

                                              LATLNG     NODE_ID  NODE_TYPE  \
0  Point (128.77178990035196193 35.13843563163282...  4180125501        104   
1  Point (128.71090530311320776 35.15963399360245...  4180125401        104   
2  Point (128.70789181712055438 35.16006572439530...  4180125301        101   
3  Point (128.70565007128658408 35.16104831672714...  4180125201        101   
4  Point (128.70337362719561725 35.16247414942506...  4180125101        101   
5  Point (128.70501074023567867 35.15994677274557...  4180125001        101   
6  Point (128.68139462430298181 35.16547661478314...  4180124901        101   
7  Point (128.67461858078345927 35.16442896911259...  4180124801        104   
8  Point (128.66942154882076466 35.16562097551557...  4180124701        104   
9  Point (128.66906606609961727 35.1658126275849412)  4180124601        104   

  NODE_NAME  TURN_P REMARK  
0      진해TG       0    NaN  
1      우남목장       0    NaN  
2     석리소류지       1    NaN  
3    레스트125   

In [24]:
tmp = list(map(lambda x: x.split(' ') , df['LATLNG']))

lng = [float(x[1][1:]) for x in tmp]
lat = [float(x[2][:-1]) for x in tmp]



df_latlng = pd.DataFrame({'NODE_ID': df['NODE_ID'], 'LNG': lng, 'LAT': lat})

print(df_latlng.head(10))

      NODE_ID         LNG        LAT
0  4180125501  128.771790  35.138436
1  4180125401  128.710905  35.159634
2  4180125301  128.707892  35.160066
3  4180125201  128.705650  35.161048
4  4180125101  128.703374  35.162474
5  4180125001  128.705011  35.159947
6  4180124901  128.681395  35.165477
7  4180124801  128.674619  35.164429
8  4180124701  128.669422  35.165621
9  4180124601  128.669066  35.165813


In [25]:
PATH_2 = r"C:\Users\Chai\.Neo4jDesktop\neo4jDatabases\database-a4494c5c-be01-475f-9407-58c9feb17dee\installation-4.2.1\import\MOCT_NODE_LATLNG_2.csv"

df_latlng.to_csv(PATH_2, sep = ',', index = False)

In [ ]:
## Very Important!!
## Create an index on node_id of KsNode
CREATE INDEX ON :KsNode(node_id)

In [ ]:
with driver.session() as session:
    session.run("""
            LOAD CSV WITH HEADERS FROM 'file:///MOCT_NODE_LATLNG_2.csv' AS row
            MATCH (n: KsNode {node_id: row['NODE_ID']})
            SET n.latlng = point({longitude: toFloat(row['LNG']), latitude: toFloat(row['LAT'])})
            """)